# Preliminaries

## Dependencies

In [1]:
!pip install amplify textualheatmap captum


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
!which python
!python --version

/home/mila/j/jerry.huang/main_envs/amplify_env/bin/python
Python 3.10.11


In [3]:
!pip install Cython
!pip install textualheatmap


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [4]:
import torch
from captum.attr import configure_interpretable_embedding_layer
from utils import load_pickle_dataset, load_from_hf, load_from_mila

/home/mila/j/jerry.huang/main_envs/amplify_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/mila/j/jerry.huang/main_envs/amplify_env/lib/python3.10/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


[2024-11-25 14:11:01,247] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [5]:
# Model
source = "mila"
model_name = "AMPLIFY350M"
model_path = "/network/projects/drugdiscovery/AMPLIFY/checkpoints/AMPLIFY_350M/pytorch_model.pt"
tokenizer_path = None 
config_path = "AMPLIFY_350_config.yaml"
device = "cuda"
compile = False
fp16 = True

## Load Model and Tokenizer

In [6]:
# Get model and tokenizer
if source == "hf":
    model, tokenizer = load_from_hf(model_path, tokenizer_path, fp16=fp16)
    interpretable_embedding = configure_interpretable_embedding_layer(model, "esm.embeddings.word_embeddings")
    bos_id, mask_id, eos_id = tokenizer.cls_token_id, tokenizer.mask_token_id, tokenizer.eos_token_id
elif source == "mila":
    model, tokenizer = load_from_mila(model_path, config_path)
    interpretable_embedding = configure_interpretable_embedding_layer(model, "encoder")
    bos_id, mask_id, eos_id = tokenizer.bos_token_id, tokenizer.mask_token_id, tokenizer.eos_token_id
else:
    raise Exception("Only 'hf' and 'mila' sources are supported, not {source}.")
model.to(device)

/home/mila/j/jerry.huang/main_envs/amplify_env/lib/python3.10/site-packages/captum/attr/_models/base.py:191: UserWarning: In order to make embedding layers more interpretable they will be replaced with an interpretable embedding layer which wraps the original embedding layer and takes word embedding vectors as inputs of the forward function. This allows us to generate baselines for word embeddings and compute attributions for each embedding dimension. The original embedding layer must be set back by calling `remove_interpretable_embedding_layer` function after model interpretation is finished. 
  warnings.warn(


AMPLIFY(
  (encoder): InterpretableEmbeddingBase(
    (embedding): Embedding(27, 960, padding_idx=0)
  )
  (transformer_encoder): ModuleList(
    (0-31): 32 x EncoderBlock(
      (q): Linear(in_features=960, out_features=960, bias=False)
      (k): Linear(in_features=960, out_features=960, bias=False)
      (v): Linear(in_features=960, out_features=960, bias=False)
      (wo): Linear(in_features=960, out_features=960, bias=False)
      (resid_dropout): Dropout(p=0, inplace=False)
      (ffn): SwiGLU(
        (w12): Linear(in_features=960, out_features=5120, bias=False)
        (w3): Linear(in_features=2560, out_features=960, bias=False)
      )
      (attention_norm): RMSNorm()
      (ffn_norm): RMSNorm()
      (ffn_dropout): Dropout(p=0, inplace=False)
    )
  )
  (layer_norm_2): RMSNorm()
  (decoder): Linear(in_features=960, out_features=27, bias=True)
)

## Load Dataset

In [171]:
import itertools
from utils import load_csv_dataset

# Dataset
n_proteins = 1_000_000

dataset = "UniProt"

if dataset == "CASP":
    data_name = "CASP14"
    data_path = "casp14.pickle"
    max_length = 2048 # AMPLIFY was trained with a context length of 512

    # Load
    output_file = "../outputs/AMPLIFY_Attribution_MaskedKL.csv"

    # Load dataset
    labels, proteins, dist_matrices = load_pickle_dataset(data_path, n_proteins, max_length)

    full_dataset = list(zip(labels, proteins))
    
    for (label, protein) in itertools.islice(full_dataset, 0, 5):
        print(label, protein)
    
else:
    # Dataset
    data_name = "UniProt"
    data_path = "uniprot_dev.csv"
    
    # Log
    output_file = "../outputs/AMPLIFY_UniProt_MaskedKL.csv"
    
    # Prepare the dataset
    full_dataset = load_csv_dataset(data_path, n_proteins)
    
    # Generator that, for each protein, tokenize, mask each residue, and batch
    # def batch_tokenize_mask(dataset, tokenizer, batch_size):
    #     for label, protein in dataset:
    #         x = torch.as_tensor(tokenizer.encode(protein, max_length=512, truncation=True))
    #         x = x.repeat(x.size(0), 1)
    #         y = torch.where(torch.eye(x.size(0), dtype=torch.bool), x, -100)
    #         x = torch.where(torch.eye(x.size(0), dtype=torch.bool), tokenizer.mask_token_id, x)
    #         for _x, _y in zip(torch.split(x, batch_size, 0), torch.split(y, batch_size, 0)):
    #             yield (label, _x, _y)
    
    for (label, protein) in itertools.islice(full_dataset, 0, 5):
        print(label, protein)
    

>A6I415|A6I415_RAT MFTVMTRQPCEQAGFRALSRTPAIVTLVVLLVSIVVLVTLTLIQIHHPQVLSPGLKYGIVLDAGSSRTTVYVYQWPAEKENNTGVVSQTFRCSVKGSGISSYENNPQDAPKAFEDCMLKVKEQVPKHLHESTRVYLGATAGMRLLRLQNETAAHEVLESIQSYFKSQPFDFRGAQIISGQEEGVYGWITANYIMGNFLEKNLWHMWVHPHGVDTTGALDLGGASTQISFVSEEKMEPNASDTVQVSLYGYTYTLYTHSFQCYGRNEAEKKFLAMLLQSPSTDANISNPCYPHGYSTTFTMGHVFGSLCTEKQRPKSYNPSDTITFTGTGDPQLCREKVASVFDFSACQEQDACSFDGIYQPKVQGPFVAFAGFYYTASALNLSGSFSLTSFNDSSWDFCRHTWSELPSLLPRFDETYARSYCFSAHYIYHLLINGYKFTEATWPQIRFEKEVGNSSIAWSLGYMLSLTNQIPAGSPLIQLPIQPPVFMGVLAFFTAIALLCLAFLFYLCSAFRTKERSENAFDQAVDSD
>Q9NVV2|CS073_HUMAN MRLKVGFQGGGCFRKDALCLEGGVSARWARAPHSAPLRPPRELHAAPPPATPTQTVVRPAGFPRRTRLMVRSAPPTQRPPTGSGCVSGLWRKGLGLRPQTLLRVGSVVLSSAPALRPRLGPCLRPPPSD
>O00628|PEX7_HUMAN MSAVCGGAARMLRTPGRHGYAAEFSPYLPGRLACATAQHYGIAGCGTLLILDPDEAGLRLFRSFDWNDGLFDVTWSENNEHVLITCSGDGSLQLWDTAKAAGPLQVYKEHAQEVYSVDWSQTRGEQLVVSGSWDQTVKLWDPTVGKSLCTFRGHESIIYSTIWSPHIPGCFASASGDQTLRIWDVKAAGVRIVIPAHQAEILSCDWCKYNENLLVTGAVDCSLRGWDLRNVRQPVFELLGHTYAIRRVKFSPFHASVLASCSYDFTVRFWNFSKPDSLLETV

In [172]:
print(len(full_dataset))

10000


# Method

## Test with one protein first

In [176]:
import itertools
import torch.nn.functional as F

# TF32 and FP16

span_probability = None
span_max = 1
mask_probability = 0.15

# span_probability = 0.15
# span_max = 10
# mask_probability = 0.15

exclude_special_tokens_replacement = False
replacement_ids = torch.ones((len(tokenizer)))
if exclude_special_tokens_replacement:
    for i in tokenizer.special_token_ids:
        replacement_ids[i] = 0

torch.manual_seed(42)

num_passes = 1

with torch.autocast(device_type=device, dtype=torch.float16, enabled=fp16):
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cudnn.allow_tf32 = True
    
    if dataset == "CASP":
        label, protein = labels[0], proteins[0]
    else:
        label, protein = full_dataset[79][0], full_dataset[79][1]
       
    # Tokenize the protein and decode to get the special tokens
    x = torch.as_tensor(tokenizer.encode(protein)).to(torch.long)
    protein = tokenizer.decode(x, skip_special_tokens=False).split()

    for _ in range(num_passes):
    
        masked_ids = torch.full(size=(len(protein),), fill_value=False, dtype=torch.bool)
        
        # Compute the padding, <bos> and <eos> masks
        pad_mask = x == tokenizer.pad_token_id
        bos_mask = x == tokenizer.bos_token_id
        eos_mask = x == tokenizer.eos_token_id
    
        # MLM
        if span_probability is None or span_max is None or span_probability == 0 or span_max == 1:
            probability_matrix = torch.full(x.shape, mask_probability)
            probability_matrix.masked_fill_(pad_mask | bos_mask | eos_mask, value=0.0)
            masked_ids = torch.bernoulli(probability_matrix).bool()
    
        # Span masking
        else:
            uniform_dist = torch.distributions.uniform.Uniform(0, len(protein))
            geometric_dist = torch.distributions.geometric.Geometric(span_probability)
            while torch.sum(masked_ids) / len(protein) < mask_probability:
                span_start = int(uniform_dist.sample().item())
                span_length = int(min(geometric_dist.sample().item(), span_max - 1, len(protein) - span_start))
                masked_ids[span_start : span_start + span_length + 1] = True
                # Unmask the padding, <bos> and <eos> tokens (note that padding should not be necessary)
                masked_ids = masked_ids & ~pad_mask & ~bos_mask & ~eos_mask
    
        y = x.clone()
        
        # 80% of the time, the masked input tokens are replaced with <MASK>
        replaced_ids = torch.bernoulli(torch.full(y.shape, 0.8)).bool() & masked_ids
        x[replaced_ids] = tokenizer.mask_token_id
    
        # 10% of the time, the masked input tokens are replaced with a random word
        random_ids = torch.bernoulli(torch.full(y.shape, 0.5)).bool() & masked_ids & ~replaced_ids
        random_words = torch.multinomial(replacement_ids, torch.numel(x), replacement=True).view(x.size())
        x[random_ids] = random_words[random_ids]
    
        reference_input = x.unsqueeze(0)
        reference_input = reference_input.to(device)
        reference_embeddings = interpretable_embedding.indices_to_embeddings(reference_input)
        print(reference_embeddings.shape)
        reference_output = model(reference_embeddings).logits
        
        mask_indices = masked_ids.nonzero()
        
        test_input = reference_input.repeat(mask_indices.size(0), 1)
        for index, value in enumerate(mask_indices):
            test_input[index, value] = y[value]
    
        assert test_input[0, mask_indices[0][0]] == y[mask_indices[0][0]], "Masking isn't correct!"
    
        test_embeddings = interpretable_embedding.indices_to_embeddings(test_input)
        test_output = model(test_embeddings).logits
    
        mask_indices = mask_indices.squeeze()
        
        pairs = itertools.permutations(torch.arange(len(mask_indices)).detach().tolist(), 2)

        tot_distance = len(protein)
    
        for (modified_index, target_index) in pairs:

            distance = abs(mask_indices[target_index] - mask_indices[modified_index]).item()
            
            original_logits = reference_output[0][mask_indices[target_index]]
            new_logits = test_output[modified_index][mask_indices[target_index]]
    
            absolute_difference = torch.mean(torch.abs(original_logits - new_logits)).detach().item()
    
            log_softmax_reference = F.log_softmax(original_logits, dim=0)
            softmax_targets = F.softmax(new_logits, dim=0)
    
            kl_div = F.kl_div(log_softmax_reference, softmax_targets, reduction="none").mean().detach().item()
    
            print(label, model_name, mask_indices.tolist(), mask_indices[modified_index].item(), mask_indices[target_index].item(), absolute_difference, kl_div, distance, tot_distance)

        break

torch.Size([1, 2803, 960])


AssertionError: 

In [219]:
import itertools
import torch.nn.functional as F
import pickle
from pathlib import Path
from tqdm import tqdm
import traceback 

def get_dataset(dataset, num_passes: int = 1):
    for i in range(num_passes):
        for sample in dataset:
            yield sample

@torch.no_grad()
def compute_attributions(
    model,
    tokenizer,
    proteins,
    device: torch.device,
    save_file: Path,
    chunk_size: int = 128,
    mask_probability: float = 0.15,
    span_probability: float = None,
    span_max: int = 1,
    masked_only: bool = True,
    exclude_special_tokens_replacement: bool = False,
    fp16: bool = True,
    max_length: int = None
):  
    # TF32 and FP16
    
    replacement_ids = torch.ones((len(tokenizer)))
    if exclude_special_tokens_replacement:
        for i in tokenizer.special_token_ids:
            replacement_ids[i] = 0

    # file = open(save_file, 'wb')
    # try:
    
    with open(save_file, 'wb') as file:
        with torch.autocast(device_type=device, dtype=torch.bfloat16, enabled=fp16):
            torch.backends.cuda.matmul.allow_tf32 = True
            torch.backends.cudnn.allow_tf32 = True
            
            for i, (label, protein) in enumerate(proteins):
               
                # Tokenize the protein and decode to get the special tokens
                x = torch.as_tensor(tokenizer.encode(protein)).to(torch.long)
                protein = tokenizer.decode(x, skip_special_tokens=False).split()

                if len(protein) > max_length:
                    continue
                
                masked_ids = torch.full(size=(len(protein),), fill_value=False, dtype=torch.bool)
                
                # Compute the padding, <bos> and <eos> masks
                pad_mask = x == tokenizer.pad_token_id
                bos_mask = x == tokenizer.bos_token_id
                eos_mask = x == tokenizer.eos_token_id
            
                # MLM
                if span_probability is None or span_max is None or span_probability == 0 or span_max == 1:
                    probability_matrix = torch.full(x.shape, mask_probability)
                    probability_matrix.masked_fill_(pad_mask | bos_mask | eos_mask, value=0.0)
                    masked_ids = torch.bernoulli(probability_matrix).bool()
            
                # Span masking
                else:
                    uniform_dist = torch.distributions.uniform.Uniform(0, len(protein))
                    geometric_dist = torch.distributions.geometric.Geometric(span_probability)
                    while torch.sum(masked_ids) / len(protein) < mask_probability:
                        span_start = int(uniform_dist.sample().item())
                        span_length = int(min(geometric_dist.sample().item(), span_max - 1, len(protein) - span_start))
                        masked_ids[span_start : span_start + span_length + 1] = True
                        # Unmask the padding, <bos> and <eos> tokens (note that padding should not be necessary)
                        masked_ids = masked_ids & ~pad_mask & ~bos_mask & ~eos_mask
            
                y = x.clone()
    
                # Replace only with mask
                if masked_only:
                    x[masked_ids] = tokenizer.mask_token_id
    
                # Replace with both mask and random words
                else:
                    # 80% of the time, the masked input tokens are replaced with <MASK>
                    replaced_ids = torch.bernoulli(torch.full(y.shape, 0.8)).bool() & masked_ids
                    x[replaced_ids] = tokenizer.mask_token_id
                
                    # 10% of the time, the masked input tokens are replaced with a random word
                    random_ids = torch.bernoulli(torch.full(y.shape, 0.5)).bool() & masked_ids & ~replaced_ids
                    random_words = torch.multinomial(replacement_ids, torch.numel(x), replacement=True).view(x.size())
                    x[random_ids] = random_words[random_ids]
    
                # Get reference output (relevant positions are replaced)
                reference_input = x.unsqueeze(0)
                reference_input = reference_input.to(device)
                reference_embeddings = interpretable_embedding.indices_to_embeddings(reference_input)
                reference_output = model(reference_embeddings).logits
    
                # Get masked outputs
                mask_indices = masked_ids.nonzero()
    
                # Create test sequences (each sequence has a single mask value replaced with its real value)
                test_input = reference_input.repeat(mask_indices.size(0), 1)
                for index, value in enumerate(mask_indices):
                    test_input[index, value] = y[value]
    
                # Assert that our replacement matches
                assert test_input[0, mask_indices[0][0]] == y[mask_indices[0][0]], "Masking isn't correct!"
    
                # Create embeddings for the testing seqeunces and get the logits from a forward pass
                test_embeddings = interpretable_embedding.indices_to_embeddings(test_input)
                test_output = model(test_embeddings).logits
    
                # Compute attributions in terms of the KL divergence and the absolute change in logits
                mask_indices = mask_indices.squeeze()

                if mask_indices.shape[0] <= 2:
                    print(f"Skipped protein {label} with length {len(protein)}.")
                
                pairs = itertools.permutations(torch.arange(mask_indices.shape[0]).detach().tolist(), 2)
        
                tot_distance = len(protein)
    
                pass_data = {
                    'label': label,
                    'model': model_name,
                    'mask_indices': mask_indices.tolist(),
                    'masked_index': [],
                    'target_index': [],
                    'absolute_difference': [], 
                    'kl_div': [], 
                    'distance': [], 
                    'tot_distance': tot_distance
                }
            
                for (modified_index, target_index) in pairs:
        
                    distance = abs(mask_indices[target_index] - mask_indices[modified_index]).item()
                    
                    original_logits = reference_output[0][mask_indices[target_index]]
                    new_logits = test_output[modified_index][mask_indices[target_index]]
            
                    absolute_difference = torch.mean(torch.abs(original_logits - new_logits)).detach().item()
            
                    log_softmax_reference = F.log_softmax(original_logits, dim=0)
                    softmax_targets = F.softmax(new_logits, dim=0)
            
                    kl_div = F.kl_div(log_softmax_reference, softmax_targets, reduction="none").mean().detach().item()
    
                    pass_data['masked_index'].append(modified_index)
                    pass_data['target_index'].append(target_index)
                    pass_data['absolute_difference'].append(absolute_difference)
                    pass_data['kl_div'].append(kl_div)
                    pass_data['distance'].append(distance)
                    
                pickle.dump(pass_data, file, protocol=pickle.HIGHEST_PROTOCOL)
    # except:
    #     file.close()
    #     traceback.print_exc() 
    #     return

In [220]:
! ls
# ! rm -r test.pkl

AMPLIFY_350_config.yaml      feature_ablation.ipynb	test.pkl
attribution.ipynb	     masked_influence.ipynb	uniprot_dev.csv
casp14.pickle		     projection_proteins.ipynb	uniprot_dev.fasta
compute_masked_influence.py  projection_residues.ipynb	uniprot_dev.fasta.zip
contact_prediction.ipynb     pseudo_ppl.ipynb		uniprot.fasta
efficiency.ipynb	     __pycache__		utils.py


In [221]:
import itertools
from utils import load_csv_dataset
from pathlib import Path
import os

torch.manual_seed(42)

save_file = Path('./test.pkl')

# Dataset
n_proteins = 1_000_000

dataset = "UniProt"

if dataset == "CASP":
    data_name = "CASP14"
    data_path = "casp14.pickle"
    max_length = 2048 # AMPLIFY was trained with a context length of 512

    # Load dataset
    labels, proteins, dist_matrices = load_pickle_dataset(data_path, n_proteins, max_length)
    full_dataset = list(zip(labels, proteins))
    
    for (label, protein) in itertools.islice(full_dataset, 0, 5):
        print(label, protein)
    
else:
    # Dataset
    data_name = "UniProt"
    data_path = "uniprot_dev.csv"
    
    # Prepare the dataset
    full_dataset = load_csv_dataset(data_path, n_proteins)
    
    for (label, protein) in itertools.islice(full_dataset, 0, 5):
        print(label, protein)

count = 0
num_passes = 5

if os.path.exists(save_file):
    file = open(save_file, 'rb')
    
    while True:
        try:
            data = pickle.load(file)
            count += 1
        except EOFError:
            break

print(f"Skipping {count} elements.")

repeated_dataset = get_dataset(full_dataset, num_passes)
sliced_dataset = itertools.islice(repeated_dataset, count, None)

>A6I415|A6I415_RAT MFTVMTRQPCEQAGFRALSRTPAIVTLVVLLVSIVVLVTLTLIQIHHPQVLSPGLKYGIVLDAGSSRTTVYVYQWPAEKENNTGVVSQTFRCSVKGSGISSYENNPQDAPKAFEDCMLKVKEQVPKHLHESTRVYLGATAGMRLLRLQNETAAHEVLESIQSYFKSQPFDFRGAQIISGQEEGVYGWITANYIMGNFLEKNLWHMWVHPHGVDTTGALDLGGASTQISFVSEEKMEPNASDTVQVSLYGYTYTLYTHSFQCYGRNEAEKKFLAMLLQSPSTDANISNPCYPHGYSTTFTMGHVFGSLCTEKQRPKSYNPSDTITFTGTGDPQLCREKVASVFDFSACQEQDACSFDGIYQPKVQGPFVAFAGFYYTASALNLSGSFSLTSFNDSSWDFCRHTWSELPSLLPRFDETYARSYCFSAHYIYHLLINGYKFTEATWPQIRFEKEVGNSSIAWSLGYMLSLTNQIPAGSPLIQLPIQPPVFMGVLAFFTAIALLCLAFLFYLCSAFRTKERSENAFDQAVDSD
>Q9NVV2|CS073_HUMAN MRLKVGFQGGGCFRKDALCLEGGVSARWARAPHSAPLRPPRELHAAPPPATPTQTVVRPAGFPRRTRLMVRSAPPTQRPPTGSGCVSGLWRKGLGLRPQTLLRVGSVVLSSAPALRPRLGPCLRPPPSD
>O00628|PEX7_HUMAN MSAVCGGAARMLRTPGRHGYAAEFSPYLPGRLACATAQHYGIAGCGTLLILDPDEAGLRLFRSFDWNDGLFDVTWSENNEHVLITCSGDGSLQLWDTAKAAGPLQVYKEHAQEVYSVDWSQTRGEQLVVSGSWDQTVKLWDPTVGKSLCTFRGHESIIYSTIWSPHIPGCFASASGDQTLRIWDVKAAGVRIVIPAHQAEILSCDWCKYNENLLVTGAVDCSLRGWDLRNVRQPVFELLGHTYAIRRVKFSPFHASVLASCSYDFTVRFWNFSKPDSLLETV

In [222]:
compute_attributions(
    model=model,
    tokenizer=tokenizer,
    proteins=sliced_dataset,
    device=device,
    save_file=save_file,
    max_length=model.config.max_length
)

KeyboardInterrupt: 

In [223]:
! ls -l -a

total 434714
drwxr-x---+  4 jerry.huang jerry.huang        19 Nov 26 17:25 .
drwxr-x---+ 11 jerry.huang jerry.huang        13 Oct 21 13:51 ..
-rw-r-----+  1 jerry.huang jerry.huang      2274 Sep 30 14:50 AMPLIFY_350_config.yaml
-rw-r-----+  1 jerry.huang jerry.huang   3423722 Nov 18 20:45 attribution.ipynb
-rw-r-----+  1 jerry.huang jerry.huang 410266502 Sep 25 14:18 casp14.pickle
-rw-r-----+  1 jerry.huang jerry.huang     16244 Nov 26 17:23 compute_masked_influence.py
-rw-r-----+  1 jerry.huang jerry.huang    253731 Sep 30 14:14 contact_prediction.ipynb
-rw-r-----+  1 jerry.huang jerry.huang      5362 Sep 30 14:14 efficiency.ipynb
-rw-r-----+  1 jerry.huang jerry.huang  10835636 Nov  8 13:56 feature_ablation.ipynb
drwxr-x---+  2 jerry.huang jerry.huang         8 Nov 26 15:10 .ipynb_checkpoints
-rw-r-----+  1 jerry.huang jerry.huang     77305 Nov 26 17:25 masked_influence.ipynb
-rw-r-----+  1 jerry.huang jerry.huang     23049 Sep 30 14:14 projection_proteins.ipynb
-rw-r-----+  1 jerry.

In [226]:
count = 0

if os.path.exists(save_file):
    file = open(save_file, 'rb')
    
    while True:
        try:
            data = pickle.load(file)
            print(data['label'])
            count += 1
        except EOFError:
            break

    file.close()

print(count)

count = 0

if os.path.exists(save_file):
    file = open(save_file, 'rb')
    
    while True:
        try:
            data = pickle.load(file)
            print(data['label'])
            count += 1
            # print(data)
        except EOFError:
            break

    file.close()

print(count)

>A0A8M9P7W8|A0A8M9P7W8_DANRE
>A0A5G2R2Q0|A0A5G2R2Q0_PIG
>Q5SIM6|Q5SIM6_THET8
3
>A0A8M9P7W8|A0A8M9P7W8_DANRE
>A0A5G2R2Q0|A0A5G2R2Q0_PIG
>Q5SIM6|Q5SIM6_THET8
3
